# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [108]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [109]:
view_sentence_range = (0, 20)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 20:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .
the lime is her least liked fruit , but the banana is my least liked .
he s

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [110]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = [[source_vocab_to_int[word] for word in n.split()] for n in source_text.split("\n")]
    target_id_text = [[target_vocab_to_int[word] for word in n.split()]+[target_vocab_to_int['<EOS>']] for n in target_text.split("\n")]
#     print("SOURCE_ID_TEXT:", source_id_text)
#     print("TARGET_ID_TEXT:", target_id_text)
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [111]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [112]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [113]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [114]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None])
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    target_sequence_length = tf.placeholder(tf.int32, [None], name="target_sequence_length")
    max_target_len = tf.reduce_max(target_sequence_length)
    source_sequence_length = tf.placeholder(tf.int32, [None], name="source_sequence_length")
#     print("INPUT:", input)
#     print("TARGETS:", targets)
#     print("LR:", learning_rate)
#     print("KEEP PROB:", keep_prob)
#     print("TSL:", target_sequence_length)
#     print("MTL:", max_target_len)
#     print("SSL:", source_sequence_length)
    return input, targets, learning_rate, keep_prob, target_sequence_length, max_target_len, source_sequence_length 


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [115]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1, 1])
    decoding_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    print("decoding_input:", decoding_input)
    return decoding_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

decoding_input: Tensor("concat:0", shape=(2, 3), dtype=int32)
Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [116]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size)
        dropout = tf.contrib.rnn.DropoutWrapper(enc_cell,output_keep_prob=keep_prob)
        return dropout

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for layer in range(num_layers)])

    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell , enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)

    return (enc_output, enc_state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [117]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)
    decode, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder, impute_finished=True, maximum_iterations=max_summary_length)
    
    return decode



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [118]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    
    with tf.variable_scope("decode", reuse=True):

        start_of_sequence_id = target_vocab_to_int['<GO>']
        end_of_sequence_id = target_vocab_to_int['<EOS>']

    
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')


    
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens,
                                                                end_of_sequence_id)
    
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       inference_helper,
                                                       encoder_state,
                                                       output_layer)
    
    inference_decoder_output, _= tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                   impute_finished=True,
                                                                   maximum_iterations= max_target_sequence_length)
    
    return inference_decoder_output
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [119]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    start_of_sequence_id = target_vocab_to_int['<GO>']
    end_of_sequence_id = target_vocab_to_int['<EOS>']

    # embed target seq
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

        # make LSTM decoder cell
    def make_cell(rnn_size):
        return tf.contrib.rnn.LSTMCell(rnn_size)

    dec_cell = tf.contrib.rnn.DropoutWrapper(
                                    tf.contrib.rnn.MultiRNNCell(
                                    [make_cell(rnn_size) for _ in range(num_layers)]),
                                    input_keep_prob=keep_prob)
        
        # create output layer
    output_layer = Dense(target_vocab_size,
                            kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
            
    with tf.variable_scope('decode'):
            # use functions created above to get training and inference logits
            training_logits = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                                                target_sequence_length, max_target_sequence_length,
                                                output_layer, keep_prob)

    with tf.variable_scope('decode', reuse=True):
            inference_logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                                                end_of_sequence_id, max_target_sequence_length, target_vocab_size,
                                                output_layer, batch_size, keep_prob)
    return training_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [133]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  encoding_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers,
                                  keep_prob,
                                  source_sequence_length,
                                  source_vocab_size, 
                                  encoding_embedding_size)
    
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input,enc_state,
                                                       target_sequence_length, max_target_sentence_length,
                                                       rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                                                       batch_size, keep_prob, dec_embedding_size)
    
    return training_decoder_output, inference_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

decoding_input: Tensor("concat:0", shape=(64, 22), dtype=int32)
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [142]:
# Number of Epochs
epochs = 20
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 40
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 30
decoding_embedding_size = 30
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.8
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [143]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


decoding_input: Tensor("concat:0", shape=(256, ?), dtype=int32)


Batch and pad the source and target sequences

In [144]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [145]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/538 - Train Accuracy: 0.2230, Validation Accuracy: 0.3180, Loss: 5.6374
Epoch   0 Batch   20/538 - Train Accuracy: 0.2645, Validation Accuracy: 0.3166, Loss: 5.1098
Epoch   0 Batch   30/538 - Train Accuracy: 0.2426, Validation Accuracy: 0.3184, Loss: 4.5652
Epoch   0 Batch   40/538 - Train Accuracy: 0.3049, Validation Accuracy: 0.3157, Loss: 3.8986
Epoch   0 Batch   50/538 - Train Accuracy: 0.2443, Validation Accuracy: 0.3157, Loss: 3.7663
Epoch   0 Batch   60/538 - Train Accuracy: 0.2369, Validation Accuracy: 0.3157, Loss: 3.6395
Epoch   0 Batch   70/538 - Train Accuracy: 0.3032, Validation Accuracy: 0.3450, Loss: 3.4320
Epoch   0 Batch   80/538 - Train Accuracy: 0.2721, Validation Accuracy: 0.3461, Loss: 3.4602
Epoch   0 Batch   90/538 - Train Accuracy: 0.3088, Validation Accuracy: 0.3455, Loss: 3.2922
Epoch   0 Batch  100/538 - Train Accuracy: 0.2840, Validation Accuracy: 0.3528, Loss: 3.3111
Epoch   0 Batch  110/538 - Train Accuracy: 0.2912, Validation Accuracy

Epoch   1 Batch  370/538 - Train Accuracy: 0.4289, Validation Accuracy: 0.4766, Loss: 1.5260
Epoch   1 Batch  380/538 - Train Accuracy: 0.4324, Validation Accuracy: 0.4741, Loss: 1.4764
Epoch   1 Batch  390/538 - Train Accuracy: 0.4589, Validation Accuracy: 0.4851, Loss: 1.4275
Epoch   1 Batch  400/538 - Train Accuracy: 0.4615, Validation Accuracy: 0.4933, Loss: 1.4402
Epoch   1 Batch  410/538 - Train Accuracy: 0.4535, Validation Accuracy: 0.4877, Loss: 1.4169
Epoch   1 Batch  420/538 - Train Accuracy: 0.4418, Validation Accuracy: 0.4931, Loss: 1.4325
Epoch   1 Batch  430/538 - Train Accuracy: 0.4574, Validation Accuracy: 0.4998, Loss: 1.4193
Epoch   1 Batch  440/538 - Train Accuracy: 0.4510, Validation Accuracy: 0.5014, Loss: 1.4452
Epoch   1 Batch  450/538 - Train Accuracy: 0.4734, Validation Accuracy: 0.4831, Loss: 1.3520
Epoch   1 Batch  460/538 - Train Accuracy: 0.4643, Validation Accuracy: 0.4851, Loss: 1.3505
Epoch   1 Batch  470/538 - Train Accuracy: 0.4660, Validation Accuracy

Epoch   3 Batch  200/538 - Train Accuracy: 0.5996, Validation Accuracy: 0.6154, Loss: 0.7712
Epoch   3 Batch  210/538 - Train Accuracy: 0.5908, Validation Accuracy: 0.6108, Loss: 0.7561
Epoch   3 Batch  220/538 - Train Accuracy: 0.5817, Validation Accuracy: 0.6161, Loss: 0.7510
Epoch   3 Batch  230/538 - Train Accuracy: 0.5773, Validation Accuracy: 0.6120, Loss: 0.7767
Epoch   3 Batch  240/538 - Train Accuracy: 0.5740, Validation Accuracy: 0.6012, Loss: 0.7852
Epoch   3 Batch  250/538 - Train Accuracy: 0.5955, Validation Accuracy: 0.6071, Loss: 0.7597
Epoch   3 Batch  260/538 - Train Accuracy: 0.5778, Validation Accuracy: 0.6119, Loss: 0.7548
Epoch   3 Batch  270/538 - Train Accuracy: 0.5816, Validation Accuracy: 0.6206, Loss: 0.7643
Epoch   3 Batch  280/538 - Train Accuracy: 0.6217, Validation Accuracy: 0.6214, Loss: 0.7199
Epoch   3 Batch  290/538 - Train Accuracy: 0.5768, Validation Accuracy: 0.6227, Loss: 0.7536
Epoch   3 Batch  300/538 - Train Accuracy: 0.6094, Validation Accuracy

Epoch   5 Batch   30/538 - Train Accuracy: 0.6525, Validation Accuracy: 0.6600, Loss: 0.5824
Epoch   5 Batch   40/538 - Train Accuracy: 0.6710, Validation Accuracy: 0.6618, Loss: 0.5056
Epoch   5 Batch   50/538 - Train Accuracy: 0.6553, Validation Accuracy: 0.6621, Loss: 0.5588
Epoch   5 Batch   60/538 - Train Accuracy: 0.6537, Validation Accuracy: 0.6618, Loss: 0.5581
Epoch   5 Batch   70/538 - Train Accuracy: 0.6577, Validation Accuracy: 0.6694, Loss: 0.5357
Epoch   5 Batch   80/538 - Train Accuracy: 0.6406, Validation Accuracy: 0.6586, Loss: 0.5846
Epoch   5 Batch   90/538 - Train Accuracy: 0.6549, Validation Accuracy: 0.6607, Loss: 0.5549
Epoch   5 Batch  100/538 - Train Accuracy: 0.6383, Validation Accuracy: 0.6680, Loss: 0.5407
Epoch   5 Batch  110/538 - Train Accuracy: 0.6334, Validation Accuracy: 0.6737, Loss: 0.5652
Epoch   5 Batch  120/538 - Train Accuracy: 0.6682, Validation Accuracy: 0.6660, Loss: 0.5279
Epoch   5 Batch  130/538 - Train Accuracy: 0.6767, Validation Accuracy

Epoch   6 Batch  390/538 - Train Accuracy: 0.7355, Validation Accuracy: 0.6994, Loss: 0.4225
Epoch   6 Batch  400/538 - Train Accuracy: 0.7007, Validation Accuracy: 0.6998, Loss: 0.4303
Epoch   6 Batch  410/538 - Train Accuracy: 0.6955, Validation Accuracy: 0.6994, Loss: 0.4395
Epoch   6 Batch  420/538 - Train Accuracy: 0.7145, Validation Accuracy: 0.7010, Loss: 0.4340
Epoch   6 Batch  430/538 - Train Accuracy: 0.6857, Validation Accuracy: 0.7061, Loss: 0.4351
Epoch   6 Batch  440/538 - Train Accuracy: 0.7205, Validation Accuracy: 0.7047, Loss: 0.4520
Epoch   6 Batch  450/538 - Train Accuracy: 0.7143, Validation Accuracy: 0.7131, Loss: 0.4343
Epoch   6 Batch  460/538 - Train Accuracy: 0.6830, Validation Accuracy: 0.6990, Loss: 0.4187
Epoch   6 Batch  470/538 - Train Accuracy: 0.7148, Validation Accuracy: 0.7026, Loss: 0.4133
Epoch   6 Batch  480/538 - Train Accuracy: 0.7033, Validation Accuracy: 0.7013, Loss: 0.4164
Epoch   6 Batch  490/538 - Train Accuracy: 0.7037, Validation Accuracy

...

Epoch  11 Batch  410/538 - Train Accuracy: 0.8512, Validation Accuracy: 0.8290, Loss: 0.2028
Epoch  11 Batch  420/538 - Train Accuracy: 0.8635, Validation Accuracy: 0.8297, Loss: 0.1894
Epoch  11 Batch  430/538 - Train Accuracy: 0.8457, Validation Accuracy: 0.8279, Loss: 0.1938
Epoch  11 Batch  440/538 - Train Accuracy: 0.8447, Validation Accuracy: 0.8320, Loss: 0.2120
Epoch  11 Batch  450/538 - Train Accuracy: 0.8181, Validation Accuracy: 0.8295, Loss: 0.2014
Epoch  11 Batch  460/538 - Train Accuracy: 0.8287, Validation Accuracy: 0.8253, Loss: 0.1911
Epoch  11 Batch  470/538 - Train Accuracy: 0.8531, Validation Accuracy: 0.8292, Loss: 0.1809
Epoch  11 Batch  480/538 - Train Accuracy: 0.8506, Validation Accuracy: 0.8242, Loss: 0.1907
Epoch  11 Batch  490/538 - Train Accuracy: 0.8469, Validation Accuracy: 0.8343, Loss: 0.1882
Epoch  11 Batch  500/538 - Train Accuracy: 0.8773, Validation Accuracy: 0.8283, Loss: 0.1676
Epoch  11 Batch  510/538 - Train Accuracy: 0.8465, Validation Accuracy

Epoch  13 Batch  240/538 - Train Accuracy: 0.8695, Validation Accuracy: 0.8635, Loss: 0.1472
Epoch  13 Batch  250/538 - Train Accuracy: 0.8658, Validation Accuracy: 0.8587, Loss: 0.1525
Epoch  13 Batch  260/538 - Train Accuracy: 0.8486, Validation Accuracy: 0.8505, Loss: 0.1542
Epoch  13 Batch  270/538 - Train Accuracy: 0.8549, Validation Accuracy: 0.8493, Loss: 0.1512
Epoch  13 Batch  280/538 - Train Accuracy: 0.8670, Validation Accuracy: 0.8521, Loss: 0.1425
Epoch  13 Batch  290/538 - Train Accuracy: 0.8678, Validation Accuracy: 0.8532, Loss: 0.1431
Epoch  13 Batch  300/538 - Train Accuracy: 0.8642, Validation Accuracy: 0.8457, Loss: 0.1480
Epoch  13 Batch  310/538 - Train Accuracy: 0.8930, Validation Accuracy: 0.8603, Loss: 0.1550
Epoch  13 Batch  320/538 - Train Accuracy: 0.8603, Validation Accuracy: 0.8583, Loss: 0.1423
Epoch  13 Batch  330/538 - Train Accuracy: 0.8806, Validation Accuracy: 0.8533, Loss: 0.1384
Epoch  13 Batch  340/538 - Train Accuracy: 0.8500, Validation Accuracy

Epoch  15 Batch   70/538 - Train Accuracy: 0.8856, Validation Accuracy: 0.8739, Loss: 0.1174
Epoch  15 Batch   80/538 - Train Accuracy: 0.8783, Validation Accuracy: 0.8707, Loss: 0.1361
Epoch  15 Batch   90/538 - Train Accuracy: 0.8966, Validation Accuracy: 0.8732, Loss: 0.1296
Epoch  15 Batch  100/538 - Train Accuracy: 0.9064, Validation Accuracy: 0.8823, Loss: 0.1111
Epoch  15 Batch  110/538 - Train Accuracy: 0.8578, Validation Accuracy: 0.8855, Loss: 0.1237
Epoch  15 Batch  120/538 - Train Accuracy: 0.9062, Validation Accuracy: 0.8745, Loss: 0.1055
Epoch  15 Batch  130/538 - Train Accuracy: 0.8986, Validation Accuracy: 0.8704, Loss: 0.1153
Epoch  15 Batch  140/538 - Train Accuracy: 0.8674, Validation Accuracy: 0.8816, Loss: 0.1393
Epoch  15 Batch  150/538 - Train Accuracy: 0.8768, Validation Accuracy: 0.8814, Loss: 0.1206
Epoch  15 Batch  160/538 - Train Accuracy: 0.8564, Validation Accuracy: 0.8674, Loss: 0.1093
Epoch  15 Batch  170/538 - Train Accuracy: 0.8698, Validation Accuracy

Epoch  16 Batch  430/538 - Train Accuracy: 0.9008, Validation Accuracy: 0.8935, Loss: 0.0973
Epoch  16 Batch  440/538 - Train Accuracy: 0.8867, Validation Accuracy: 0.8975, Loss: 0.1139
Epoch  16 Batch  450/538 - Train Accuracy: 0.8919, Validation Accuracy: 0.8951, Loss: 0.1127
Epoch  16 Batch  460/538 - Train Accuracy: 0.8845, Validation Accuracy: 0.8938, Loss: 0.1027
Epoch  16 Batch  470/538 - Train Accuracy: 0.9010, Validation Accuracy: 0.8844, Loss: 0.0943
Epoch  16 Batch  480/538 - Train Accuracy: 0.9012, Validation Accuracy: 0.8851, Loss: 0.1032
Epoch  16 Batch  490/538 - Train Accuracy: 0.9103, Validation Accuracy: 0.9016, Loss: 0.0987
Epoch  16 Batch  500/538 - Train Accuracy: 0.9089, Validation Accuracy: 0.9114, Loss: 0.0810
Epoch  16 Batch  510/538 - Train Accuracy: 0.8990, Validation Accuracy: 0.9004, Loss: 0.0959
Epoch  16 Batch  520/538 - Train Accuracy: 0.8826, Validation Accuracy: 0.8975, Loss: 0.1011
Epoch  16 Batch  530/538 - Train Accuracy: 0.8877, Validation Accuracy

Epoch  18 Batch  260/538 - Train Accuracy: 0.8888, Validation Accuracy: 0.8968, Loss: 0.0907
Epoch  18 Batch  270/538 - Train Accuracy: 0.9027, Validation Accuracy: 0.9096, Loss: 0.0834
Epoch  18 Batch  280/538 - Train Accuracy: 0.9103, Validation Accuracy: 0.9196, Loss: 0.0783
Epoch  18 Batch  290/538 - Train Accuracy: 0.9055, Validation Accuracy: 0.9137, Loss: 0.0775
Epoch  18 Batch  300/538 - Train Accuracy: 0.9116, Validation Accuracy: 0.9160, Loss: 0.0835
Epoch  18 Batch  310/538 - Train Accuracy: 0.9314, Validation Accuracy: 0.9132, Loss: 0.0921
Epoch  18 Batch  320/538 - Train Accuracy: 0.9183, Validation Accuracy: 0.9139, Loss: 0.0828
Epoch  18 Batch  330/538 - Train Accuracy: 0.9118, Validation Accuracy: 0.9031, Loss: 0.0816
Epoch  18 Batch  340/538 - Train Accuracy: 0.8875, Validation Accuracy: 0.9137, Loss: 0.0813
Epoch  18 Batch  350/538 - Train Accuracy: 0.9116, Validation Accuracy: 0.9174, Loss: 0.0961
Epoch  18 Batch  360/538 - Train Accuracy: 0.8854, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [146]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [147]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [195]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sent_list = []
    words = sentence.lower().split()
    for w in words:
        if(w in vocab_to_int):
            sent_list.append(vocab_to_int[w])
        else:
            sent_list.append(vocab_to_int['<UNK>'])

    return sent_list
    
    
    
        

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [196]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [144, 200, 58, 107, 87, 224, 164]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [108, 208, 195, 293, 251, 339, 301, 241, 1]
  French Words: il a vu un vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.